### Recommendations with MovieTweetings: Getting to Know The Data

Throughout this lesson, you will be working with the [MovieTweetings Data](https://github.com/sidooms/MovieTweetings/tree/master/recsyschallenge2014).  To get started, you can read more about this project and the dataset from the [publication here](http://crowdrec2013.noahlab.com.hk/papers/crowdrec2013_Dooms.pdf).

**Note:** There are solutions to each of the notebooks available by hitting the orange jupyter logo in the top left of this notebook.  Additionally, you can watch me work through the solutions on the screencasts that follow each workbook. 

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.

 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('original_movies.dat',
                     delimiter='::',
                     header=None,
                     names=['movie_id', 'movie', 'genre'],
                     dtype={'movie_id': object}, engine='python')

reviews = pd.read_csv('original_ratings.dat',
                      delimiter='::',
                      header=None,
                      names=['user_id', 'movie_id', 'rating', 'timestamp'],
                      dtype={'movie_id': object, 'user_id': object, 'timestamp': object},
                      engine='python')

# Reduce the size reviews dataset
reviews = reviews.loc[:100000,:]

In [2]:
movies.head()

,movie_id,movie,genre
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short
1,0000010,La sortie des usines Lumière (1895),Documentary|Short
2,0000012,The Arrival of a Train (1896),Documentary|Short
3,25,The Oxford and Cambridge University Boat Race ...,NaN
4,0000091,Le manoir du diable (1896),Short|Horror


In [3]:
reviews.head()

,user_id,movie_id,rating,timestamp
0,1,0114508,8,1381006850
1,2,0208092,5,1586466072
2,2,0358273,9,1579057827
3,2,10039344,5,1578603053
4,2,6751668,9,1578955697


#### 1. Take a Look At The Data 

Take a look at the data and use your findings to fill in the dictionary below with the correct responses to show your understanding of the data.

In [4]:
# Use this cell to find the following information:
# number of movies
num_movies = len(movies)
print(f'The number of movies is {num_movies}')
# number of ratings
num_rating = len(reviews)
print(f'The number of ratings is {num_rating}')
# number of unique users
num_users = len(reviews.user_id.value_counts())
print(f'The number of users is {num_users}')
# number of missing ratings
missing_ratings = reviews.rating.isnull().sum()
print(f'The number of missing ratings is {missing_ratings}')
# the `average`, `min`, and `max` ratings given
avg_rating = np.mean(reviews.rating)
print(f'The average rating is {avg_rating}')
min_rating = np.min(reviews.rating)
print(f'The minimum rating is {min_rating}')
max_rating = np.max(reviews.rating)
print(f'The maximum rating is {max_rating}')

The number of movies is 35479
The number of ratings is 100001
The number of users is 8022
The number of missing ratings is 0
The average rating is 7.397666023339767
The minimum rating is 0
The maximum rating is 10


In [5]:
# number of different genres
num_genres = movies.genre.value_counts()
# print(f'{num_genres}')
genres = []

for val in movies.genre:
    try:
        genres.extend(val.split('|'))
    except AttributeError:
        pass

genres = set(genres)
print(genres)

{'Music', 'Horror', 'Reality-TV', 'Adult', 'Action', 'Mystery', 'Crime', 'Western', 'Drama', 'Fantasy', 'News', 'Talk-Show', 'Sci-Fi', 'Romance', 'Thriller', 'Game-Show', 'Family', 'War', 'History', 'Musical', 'Adventure', 'Documentary', 'Comedy', 'Short', 'Sport', 'Biography', 'Film-Noir', 'Animation'}


In [6]:
# Use your findings to match each variable to the correct statement in the dictionary
a = 8022
b = 10
c = 7
d = 35479
e = 15
f = 0
g = 4
h = 100001
i = 28

# Guide: replace "ENTER YOUR ANSWER HERE" with the matching letter above.
# For example, 'The number of movies in the dataset': d,

dict_sol1 = {
'The number of movies in the dataset': d, 
'The number of ratings in the dataset': h,
'The number of different genres': i, 
'The number of unique users in the dataset': a, 
'The number missing ratings in the reviews dataset': f, 
'The average rating given across all ratings': c,
'The minimum rating given across all ratings': f,
'The maximum rating given across all ratings': b
}

# Check your solution
t.q1_check(dict_sol1)

That looks good to me!


#### 2. Data Cleaning

Next, we need to pull some additional relevant information out of the existing columns. 

For each of the datasets, there are a couple of cleaning steps we need to take care of:

#### Movies
* Pull the date from the title and create new column
* Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)
* Dummy column the genre with 1's and 0's for each genre

#### Reviews
* Create a date out of time stamp

You can check your results against the header of my solution by running the cell below with the **show_clean_dataframes** function.

In [8]:
# Cell for work
# Pull the date from title and create new column

movies['year'] = movies.movie.str.extract(r'(\d{4})')

# Dummy the date column with 1's and 0's for each century of 
# a movie (1800's, 1900's, and 2000's)

# converting the year column to integer
movies['year'] = movies['year'].astype(int, inplace = True)

#create bins based on century
movies['century'] = pd.cut(movies.year, bins=[1799, 1899, 1999, 2100], labels=['1800s', '1900s', '2000s'])

# create binary columns for each century
movies['1800s'] = movies['century'].map({'1800s':1, '1900s':0, '2000s':0})
movies['1900s'] = movies['century'].map({'1800s':0, '1900s':1, '2000s':0})
movies['2000s'] = movies['century'].map({'1800s':0, '1900s':0, '2000s':1})

movies.drop(columns = 'century', inplace = True)

# Dummy column the genre with 1's and 0's for each genre
genres_ohe = movies['genre'].str.get_dummies(sep = '|')
movies = pd.concat([movies, genres_ohe], axis = 1)

In [13]:
# Create a date out of time stamp
reviews['date'] = pd.to_datetime(reviews['timestamp'], unit = 's')

In [14]:
reviews.head()

,user_id,movie_id,rating,timestamp,date
0,1,0114508,8,1381006850,2013-10-05 21:00:50
1,2,0208092,5,1586466072,2020-04-09 21:01:12
2,2,0358273,9,1579057827,2020-01-15 03:10:27
3,2,10039344,5,1578603053,2020-01-09 20:50:53
4,2,6751668,9,1578955697,2020-01-13 22:48:17


In [75]:
reviews_new, movies_new = t.show_clean_dataframes()

   Unnamed: 0  user_id  movie_id  rating   timestamp                 date
0           0        1    114508       8  1381006850  2013-10-05 21:00:50
1           1        2    208092       5  1586466072  2020-04-09 21:01:12
2           2        2    358273       9  1579057827  2020-01-15 03:10:27
3           3        2  10039344       5  1578603053  2020-01-09 20:50:53
4           4        2   6751668       9  1578955697  2020-01-13 22:48:17
   Unnamed: 0  movie_id                                              movie  \
0           0         8      Edison Kinetoscopic Record of a Sneeze (1894)   
1           1        10                La sortie des usines Lumière (1895)   
2           2        12                      The Arrival of a Train (1896)   
3           3        25  The Oxford and Cambridge University Boat Race ...   
4           4        91                         Le manoir du diable (1896)   

               genre  date  1800's  1900's  2000's  Documentary  Adventure  \
0  Docume